## Initialize

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI

model="claude-sonnet-4-20250514"
#model="claude-opus-4-20250514"

client_config = {
    "type": "ttt",
    "client_type": "anthropic",
    "model": model
    }


### Generate

In [2]:
import anthropic

model = "claude-sonnet-4-20250514"
# model="claude-opus-4-20250514"

## Using anthropic client ────────────────────────────────────────────────────────────────────────

print("\n\nNormal:\n")

client = anthropic.Anthropic()
response = client.messages.create(
    model=model,
    max_tokens=1000,
    messages=[
        {
            "role": "user",
            "content": "tell me a one sentence story",
        },
    ],
)
print(f"Raw:\n{response.model_dump()}\nExtracted:\n{response.content[0].text}")

## Using OpenAI-Compatible client ───────────────────────────────────────────────────────────────────

print("\n\nPatched:\n")

from gai.llm.openai import OpenAI
client = OpenAI(client_config=client_config)
response = client.chat.completions.create(
    model=client_config["model"], 
    messages=[{"role":"user","content":"tell me a one sentence story"}]
)
print(f"Raw:\n{response.model_dump()}\nExtracted:\n{response.extract()}")

## Using OpenAI-Compatible async client ───────────────────────────────────────────────────────────────────

print("\n\nAsync:\n")

from gai.llm.openai import AsyncOpenAI
client = AsyncOpenAI(client_config=client_config)
response = await client.chat.completions.create(
    model=client_config["model"], 
    messages=[{"role":"user","content":"tell me a one sentence story"}]
)
print(f"Raw:\n{response.model_dump()}\nExtracted:\n{response.extract()}")



Normal:

Raw:
{'id': 'msg_013kK2TtwN69Vq7Vhn51jksT', 'content': [{'citations': None, 'text': 'The last person on Earth sat alone in a room, and then there was a knock at the door.', 'type': 'text'}], 'model': 'claude-sonnet-4-20250514', 'role': 'assistant', 'stop_reason': 'end_turn', 'stop_sequence': None, 'type': 'message', 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 13, 'output_tokens': 24, 'server_tool_use': None, 'service_tier': 'standard'}}
Extracted:
The last person on Earth sat alone in a room, and then there was a knock at the door.


Patched:

Raw:
{'id': 'chatcmpl-92f80755-b066-40b9-aa39-3e95ed1e5f5d', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': 'The last person on Earth sat alone in a room when suddenly, there was a knock at the door.', 'refusal': None, 'role': 'assistant', 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': None}}], 'created': 1749886919, 'mod

---

### Streaming

In [3]:
import anthropic

model = "claude-sonnet-4-20250514"
# model="claude-opus-4-20250514"

## Using anthropic client ────────────────────────────────────────────────────────────────────────

print("\n\nNormal:\n")

import json
client = anthropic.Anthropic()
response = client.messages.create(
    model=model,
    max_tokens=1000,
    messages=[
        {
            "role": "user",
            "content": "Tell me a one sentence story",
        },
    ],
    stream=True
)
chunks = []
for chunk in response:
    if chunk.type=="content_block_delta":
        chunks.append(chunk.delta.text)
print(json.dumps(chunks,indent=4))
            
## Using gai.llm.openai client ───────────────────────────────────────────────────────────────────

print("\n\nPatched:\n")

from gai.llm.openai import OpenAI
client = OpenAI(client_config=client_config)
response = client.chat.completions.create(
    model=client_config["model"], 
    messages=[{"role":"user","content":"tell me a one sentence story"}],
    stream=True
)
chunks = []
for chunk in response:
    chunk = chunk.extract()
    if (isinstance(chunk, str)):
        print(chunk, end="", flush=True)


import os
from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI

model="claude-opus-4-20250514"

client_config = {
    "type": "ttt",
    "client_type": "anthropic",
    "model": model
    }

# # Using gai.llm.openai async client ───────────────────────────────────────────────────────────────────

print("\n\nAsync:\n")

from gai.llm.openai import AsyncOpenAI
client = AsyncOpenAI(client_config=client_config)
response = await client.chat.completions.create(
    model=client_config["model"], 
    messages=[{"role":"user","content":"tell me a one sentence story"}],
    stream=True
)
chunks = []
async for chunk in response:
    if chunk:
        chunk = chunk.extract()
        if (isinstance(chunk, str)):
            print(chunk, end="", flush=True)
    




Normal:

[
    "The",
    " last person on Earth sat alone in a",
    " room, when suddenly there was a knock at the door."
]


Patched:

The last person on Earth sat alone in a room when suddenly, there was a knock at the door.

Async:

The last librarian on Earth carefully shelved the final printed book, turned off the lights, and locked the door behind her forever.

---
### Tool Call

In [4]:
import anthropic

model = "claude-sonnet-4-20250514"
# model="claude-opus-4-20250514"

## Using anthropic client ────────────────────────────────────────────────────────────────────────

print("\n\nNormal:\n")

client = anthropic.Anthropic()
response = client.messages.create(
    model=model, 
    messages=[{"role": "user", "content": "What is the current time in Singapore?"}],
    tools=[
        {
            "name": "google",
            "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
            "input_schema": {
                "type": "object",
                "properties": {
                    "search_query": {
                        "type": "string",
                        "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                    }
                },
                "required": ["search_query"]
            }
        }
    ],
    max_tokens=1000,
)
import json
print(json.dumps(response.model_dump(), indent=4))

## Using OpenAI-Compatible client ───────────────────────────────────────────────────────────────────

print("\n\nPatched:\n")

from gai.llm.openai import OpenAI
client = OpenAI(client_config=client_config)
response = client.chat.completions.create(
    model=client_config["model"],
    messages=[{"role": "user", "content": "What is the current time in Singapore?"}],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "google",
                "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "search_query": {
                            "type": "string",
                            "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                        }
                    },
                    "required": ["search_query"]
                }
            }
        }
    ]
)

# For anthropic, the result will be returned as serialized array of ContentBlocks as ChatCompletionMessage's content.

print(response.extract())

## Using OpenAI-Compatible async client ───────────────────────────────────────────────────────────────────

print("\n\nAsync:\n")

from gai.llm.openai import AsyncOpenAI
client = AsyncOpenAI(client_config=client_config)
response = await client.chat.completions.create(
    model=client_config["model"],
    messages=[{"role": "user", "content": "What is the current time in Singapore?"}],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "google",
                "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "search_query": {
                            "type": "string",
                            "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                        }
                    },
                    "required": ["search_query"]
                }
            }
        }
    ],
    tool_choice="required"
)
print(response.extract())



Normal:

{
    "id": "msg_01CXQrEPzpPQ6Y62LUgJxZhf",
    "content": [
        {
            "citations": null,
            "text": "I'll help you find the current time in Singapore by searching for this information.",
            "type": "text"
        },
        {
            "id": "toolu_01McRqM73Bswhmgrh6cX4mD9",
            "input": {
                "search_query": "current time Singapore"
            },
            "name": "google",
            "type": "tool_use"
        }
    ],
    "model": "claude-sonnet-4-20250514",
    "role": "assistant",
    "stop_reason": "tool_use",
    "stop_sequence": null,
    "type": "message",
    "usage": {
        "cache_creation_input_tokens": 0,
        "cache_read_input_tokens": 0,
        "input_tokens": 578,
        "output_tokens": 72,
        "server_tool_use": null,
        "service_tier": "standard"
    }
}


Patched:

[{'citations': None, 'text': "I'll help you find the current time in Singapore.", 'type': 'text'}, {'id': 'toolu_01SeHb

#### Parameterless Tool Call

In [ ]:
import anthropic

model = "claude-sonnet-4-20250514"
# model="claude-opus-4-20250514"

from gai.llm.openai import AsyncOpenAI
client = AsyncOpenAI(client_config={
    "client_type": "anthropic",
    "model": model,
})
response = await client.chat.completions.create(
    model=client_config["model"],
    messages=[{"role": "user", "content": "What is the current time in Singapore?"}],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_current_time",
                "description": "Get current time in Singapore",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "required": []
                }
            }
        }
    ],
    tool_choice="required"
)
print(response.extract())

[{"citations": null, "text": "I'll get the current time in Singapore for you.", "type": "text"}, {"id": "toolu_0168bhSBi2TpewxdCn2oD657", "input": {}, "name": "get_current_time", "type": "tool_use"}]


#### Streaming Tool Call

Can't think of a good reason why tool call should be streamed but here it is provided to maintain compatibility with the original API.



In [ ]:
model = "claude-sonnet-4-20250514"
# model="claude-opus-4-20250514"

# ## Using OpenAI-Compatible client ───────────────────────────────────────────────────────────────────
from rich import print as rprint
from gai.llm.openai import AsyncOpenAI
client = AsyncOpenAI(client_config=client_config)
response = await client.chat.completions.create(
    model=client_config["model"],
    messages=[{"role": "user", "content": "What is the current time in Singapore?"}],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "google",
                "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "search_query": {
                            "type": "string",
                            "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                        }
                    },
                    "required": ["search_query"]
                }
            }
        }
    ],
    stream=True,
)

# The stream starts by returning a series of text chunks forming the text_block and ends with a final Message which returns the entirety of the result
# corresponding to the result returned from an unstreamed call.
content=None
async for chunk in response:
    if chunk:
        print()
        print(chunk.extract())





I'll help

 you find the current time in Singapore.

[{'citations': None, 'text': "I'll help you find the current time in Singapore.", 'type': 'text'}, {'id': 'toolu_017b4DQuWwptdH6fJ7cYqgUu', 'input': {'search_query': 'current time in Singapore'}, 'name': 'google', 'type': 'tool_use'}]

{'type': 'finish_reason', 'finish_reason': 'tool_calls'}


When tool call is not needed.

In [1]:
model = "claude-opus-4-20250514"

# ## Using OpenAI-Compatible client ───────────────────────────────────────────────────────────────────

print("\n\nPatched:\n")

from gai.llm.openai import AsyncOpenAI
client = AsyncOpenAI(client_config=client_config)
response = await client.chat.completions.create(
    model=client_config["model"],
    messages=[{"role": "user", "content": "Tell me a one sentence story"}],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "google",
                "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "search_query": {
                            "type": "string",
                            "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                        }
                    },
                    "required": ["search_query"]
                }
            }
        }
    ],
    stream=True,
)

async for chunk in response:
    if chunk:
        print(chunk)
        print()
    




Patched:



NameError: name 'client_config' is not defined

---

### Structured Output


In [4]:

from pydantic import BaseModel
class Book(BaseModel):
    title: str
    summary: str
    author: str
    published_year: int
data = """Foundation is a science fiction novel by American writer
        Isaac Asimov. It is the first published in his Foundation Trilogy (later
        expanded into the Foundation series). Foundation is a cycle of five
        interrelated short stories, first published as a single book by Gnome Press
        in 1951. Collectively they tell the early story of the Foundation,
        an institute founded by psychohistorian Hari Seldon to preserve the best
        of galactic civilization after the collapse of the Galactic Empire."""       

import anthropic

model = "claude-opus-4-20250514"

## Using anthropic client ────────────────────────────────────────────────────────────────────────

print("\n\nNormal:\n")

client = anthropic.Anthropic()

# Use tool calls to return structured output
book_tool = {
    "name": "structured_output",
    "description": "Extract and structure book information",
    "input_schema": Book.model_json_schema()
}

response = client.messages.create(
    model=model,
    max_tokens=1000,
    temperature=0,
    tools=[book_tool],
    messages=[
        {
            "role": "user",
            "content": f"Extract structured output from this: {data}"
        }
    ]
)
print(response.model_dump_json(indent=4))

## Using OpenAI-Compatible client ───────────────────────────────────────────────────────────────────

print("\n\nPatched:\n")

from gai.llm.openai import OpenAI
client = OpenAI(client_config=client_config)
response = client.beta.chat.completions.parse(
    model=client_config["model"], 
    response_format=Book,
    messages=[{"role":"user","content":data}]
    )
print(response.model_dump_json(exclude={"extract"},indent=4))

print("\n\nExtracted:\n")
print(response.extract())

## Using OpenAI-Compatible async client ───────────────────────────────────────────────────────────────────

print("\n\nPatched:\n")

from gai.llm.openai import AsyncOpenAI
client = AsyncOpenAI(client_config=client_config)
response = await client.beta.chat.completions.parse(
    model=client_config["model"], 
    response_format=Book,
    messages=[{"role":"user","content":data}]
    )
print(response.model_dump_json(exclude={"extract"},indent=4))

print("\n\nExtracted:\n")
print(response.extract())




Normal:

{
    "id": "msg_015oMv9grVTpyJy8Pqu87GEB",
    "content": [
        {
            "citations": null,
            "text": "I'll extract the structured information from this text about the book \"Foundation\".",
            "type": "text"
        },
        {
            "id": "toolu_015qCs4okfv3CdrNQ6hBSQ8Y",
            "input": {
                "title": "Foundation",
                "author": "Isaac Asimov",
                "published_year": 1951,
                "summary": "Foundation is a science fiction novel that is the first published in the Foundation Trilogy (later expanded into the Foundation series). It consists of a cycle of five interrelated short stories that collectively tell the early story of the Foundation, an institute founded by psychohistorian Hari Seldon to preserve the best of galactic civilization after the collapse of the Galactic Empire."
            },
            "name": "structured_output",
            "type": "tool_use"
        }
    ],
    "mo

/workspaces/gai-sdk/.venv/lib/python3.10/site-packages/pydantic/main.py:519: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [input_value=[{'citations': None, 'tex...t', 'type': 'tool_use'}], input_type=list])
  return self.__pydantic_serializer__.to_json(


{
    "id": "chatcmpl-bf835a5c-af85-465e-9d71-637b9a2afb4a",
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": [
                    {
                        "citations": null,
                        "text": "I'll extract the information about the Foundation novel and return it in JSON format.",
                        "type": "text"
                    },
                    {
                        "id": "toolu_01X8Zn8J3wjkdEPzci7xpNW6",
                        "input": {
                            "title": "Foundation",
                            "summary": "Foundation is a science fiction novel that is the first published in the Foundation Trilogy (later expanded into the Foundation series). It is a cycle of five interrelated short stories, first published as a single book by Gnome Press in 1951. The stories collectively tell the early story of the Foundation

---

### Serialization

In [9]:
from typing import List, Union, Annotated
from anthropic.types import TextBlock, ToolUseBlock
from pydantic import Field,BaseModel,TypeAdapter
MessageBlock = Annotated[Union[TextBlock, ToolUseBlock],Field(discriminator="type")]

## Using Opus ───────────────────────────────────────────────────────────────────

print("\n\nOpus-4-20250514:\n")


MessageList = TypeAdapter(List[MessageBlock])
    
messages = [
    TextBlock(type="text",text="Hello, how are you?"),
    ToolUseBlock(
        id="aaa",
        type="tool_use",
        name="google",
        input={"search_query": "current weather in Singapore"}
    )]

message_list = MessageList.dump_json(messages)
print(message_list)

loaded = MessageList.validate_json(message_list)
print(loaded)
    



Opus-4-20250514:

b'[{"citations":null,"text":"Hello, how are you?","type":"text"},{"id":"aaa","input":{"search_query":"current weather in Singapore"},"name":"google","type":"tool_use"}]'
[TextBlock(citations=None, text='Hello, how are you?', type='text'), ToolUseBlock(id='aaa', input={'search_query': 'current weather in Singapore'}, name='google', type='tool_use')]
